In [43]:
import os
import torch
from torchvision import datasets
from torchvision import transforms

from utils.datasets import get_knn_evaluation_loader

In [56]:
loader, loader2 = get_knn_evaluation_loader('TinyImagenet', batch_size=16)

In [57]:
loader.dataset[0].transform

In [40]:
[l for l in loader.dataset[0].__dir__() if l[:2]!='__']

['root',
 'transform',
 'target_transform',
 'transforms',
 'loader',
 'extensions',
 'classes',
 'class_to_idx',
 'samples',
 'targets',
 'imgs',
 'make_dataset',
 'find_classes',
 '_repr_indent',
 '_format_transform_repr',
 'extra_repr',
 '_is_protocol']

In [41]:
np.unique(loader.dataset[0].targets)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [42]:
import numpy as np
np.unique(loader.dataset.classes)

AttributeError: 'tuple' object has no attribute 'classes'

In [58]:
[l for l in loader.dataset.__dir__() if l[:2]!='__']
import torchvision


In [62]:
single_transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = torchvision.datasets.ImageFolder('dataset/tiny-imagenet-200/train', transform=single_transform)

In [65]:
train_dataset

Dataset ImageFolder
    Number of datapoints: 100000
    Root location: dataset/tiny-imagenet-200/train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [ ]:
train_dir = 'dataset/tiny-imagenet-200/train'
val_dir = 'dataset/tiny-imagenet-200/val/images'

# Pre-calculated mean & std on imagenet:
# norm = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# For other datasets, we could just simply use 0.5:
# norm = transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])

print('Preparing dataset ...')

# Data augmentation (torchsample)
# torchsample doesn't really help tho...

train_data = datasets.ImageFolder(train_dir, )
val_data = datasets.ImageFolder(val_dir,)

print('Preparing data loaders ...')
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)

val_data_loader = torch.utils.data.DataLoader(val_data, batch_size=64, shuffle=True)

Preparing dataset ...
Preparing data loaders ...


In [ ]:
train_data[600][0].size

(64, 64)

In [ ]:
next(iter(train_data_loader))

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.Image.Image'>

In [ ]:
val_dir = 'dataset/tiny-imagenet-200/val'
img_dir = 'dataset/tiny-imagenet-200/val/images'

fp = open(os.path.join(val_dir, 'val_annotations.txt'), 'r')
data = fp.readlines()
val_img_dict = {}
for line in data:
    words = line.split('\t')
    val_img_dict[words[0]] = words[1]
fp.close()

# Create folder if not present and move images into proper folders
for img, folder in val_img_dict.items():
    newpath = (os.path.join(img_dir, folder))
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    if os.path.exists(os.path.join(img_dir, img)):
        os.rename(os.path.join(img_dir, img), os.path.join(newpath, img))